<a href="https://colab.research.google.com/github/RachitBansal/AppliancePower_TimeSeries/blob/master/ALL_MODELS_CLASS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
####code to  upload the data and resample the data (day wise)
from sklearn.externals import joblib
from keras.layers import Dense ,LSTM
from keras.models import Sequential 
from sklearn.metrics import mean_absolute_error,mean_squared_error
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib import pyplot
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from sklearn.metrics import r2_score as r2
import matplotlib.pyplot as plt
import tensorflow as tf
from pandas import DataFrame
from statsmodels.tsa.arima_model import ARIMA
# import r2
# import mane

import keras.backend as K

import os

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import keras.backend as K

def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true-y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

In [0]:
from keras.models import load_model
import xgboost

In [0]:
class Ensemble_pipeline():
    def __init__(self, equipment, n_members):
        self.eq = equipment
        self.n_members = n_members

    def load_arima_res(self):
        return np.load('./drive/My Drive/UkDale/ARIMA/arima_ukdale_preds_1000_eq'+str(self.eq)+'.npy')

    def fit_cnn_lstm(self,epochs=20,batch_size=128): 
        n_timesteps, n_features = self.train_x.shape[1], self.train_x.shape[2]
        model = Sequential()
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,1)))
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(RepeatVector(1))
        model.add(LSTM(200, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(100, activation='relu')))
        model.add(TimeDistributed(Dense(1)))
        # model.summary()
        model.compile(loss='mse', optimizer='adam',metrics = ["accuracy"])

        model.fit(self.train_x, self.train_y, epochs=10, batch_size=128, verbose = 1)

        return model
    
    def fit_ANN(self,epochs=20,batch_size=128):
        x_train = self.train_x.reshape(-1, 128)
        ann = Sequential()
        ann.add(Dense(256,activation = 'relu',input_dim = 128))
        ann.add(Dense(64,activation = 'relu'))
        ann.add(Dense(64,activation = 'relu'))
        ann.add(Dense(1,activation = 'relu'))
        
        ann.compile (optimizer = 'RMSprop',loss = 'mean_squared_error',metrics = ['accuracy'])
        ann.fit(x_train,self.train_y[:,i], batch_size = batch_size, epochs = epochs)
        
        return ann
    
    def fit_vanilla_lstm(self,epochs=20,batch_size=128):
        regressor = Sequential()
        regressor.add(LSTM(units = 10,activation = "relu",input_shape= (128,1)))
        regressor.add(Dense(units = 1))
        regressor.compile(optimizer = "adam",loss = "mean_squared_error")
        regressor.fit(self.train_x,self.train_y,epochs = epochs, batch_size = batch_size,verbose  = 0)
 
        return regressor
   
    def load_data(self):
        self.train_x = np.load(file='./drive/My Drive/UkDale/ukdale_'+str(self.eq)+'_x.npy')
        self.train_y = np.load(file='./drive/My Drive/UkDale/ukdale_'+str(self.eq)+'_y.npy')
        test_x = np.load(file='./drive/My Drive/UkDale/ukdale_'+str(self.eq)+'_tx.npy')
        test_y = np.load(file='./drive/My Drive/UkDale/ukdale_'+str(self.eq)+'_ty.npy')
        val_split = int(0.2*test_x.shape[0])
        self.val_x = test_x[-val_split:]
        self.val_y = test_y[-val_split:]
        self.test_x = test_x[:-val_split]
        self.test_y = test_y[:-val_split]
        print(self.val_x.shape, self.val_y.shape, self.test_x.shape, self.test_y.shape)
    
    def load_all_models(self):
        all_models = list()
        for i in range(self.n_members):
            # define filename for this ensemble
            filename = './drive/My Drive/ukdale'+str(self.eq-1)+'_model_' + str(i + 1) + '.h5'
        # load model from file
            model = load_model(filename)
        # add to list of members
            all_models.append(model)
            print('>>> loaded %s' % filename)
        return all_models

    def stacked_dataset(self, inputX):
        stackX = None
        for model in self.members:
            # make prediction
            yhat = model.predict(inputX, verbose=0)
            # stack predictions into [rows, members, probabilities]
            if stackX is None:
                stackX = yhat
            else:
                stackX = np.dstack((stackX, yhat))

        stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
        return stackX

    def fit_stacked_model(self, inputX, inputy, n_est=45, alpha = 10):
      # create dataset using ensemble
        stackedX = self.stacked_dataset(inputX)
      # fit standalone model
        self.ensemble = xgboost.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = alpha, n_estimators = n_est)
        self.ensemble.fit(stackedX, inputy)

    def stacked_prediction(self, inputX):
      # create dataset using ensemble
        stackedX = self.stacked_dataset(inputX)
      # make a prediction
        yhat = self.ensemble.predict(stackedX)
        return yhat

    def make_cnnlstm_comparison_graph(self, n_test, idx=0):
        preds = []
        lstm_preds = []
        for i in range(n_test):
            lstm_preds.append(self.members[0].predict(self.test_x[i].reshape(1,-1,1))[0][0][0])
        e_preds = self.stacked_prediction(self.test_x[idx:idx+n_test])
        a_preds = self.load_arima_res()
        preds.append(lstm_preds)            
        preds.append(e_preds)
        preds.append(a_preds[:n_test])
        print(len(preds))
        # np.save(file = './drive/My Drive/UkDale/preds_'+str(self.eq)+'.npy', arr = np.array(preds))

        plt.plot(self.test_y[idx:idx+n_test].reshape(-1), c = 'orange', label='actual')
        list_colrs = ['red', 'blue', 'red']
        labels = ['CNN-LSTM', 'XG-Boost ensemble', 'ARIMA']
        for j in range(3):
            plt.plot(preds[j], c = list_colrs[j], label=labels[j])
        plt.ylabel('Value')
        plt.xlabel('Index of Time Series Data')
        plt.legend()
        plt.show()
    
    def arima_summary(x):
        series = x[:, 0]
        series = np.array(series)
        model = ARIMA(series, order=(5,1,0))
        model_fit = model.fit(disp=0)
        print(model_fit.summary())
        # plot residual errors
        residuals = DataFrame(model_fit.resid)
        residuals.plot()
        pyplot.show()
        residuals.plot(kind='kde')
        pyplot.show()
        print(residuals.describe())
        
    def arima_predictions():
        x_test = self.test_x
        y_test = self.test_y.reshape((-1,))
        predictions = []
        k = y_test.shape[0]
        history = list(x_test[0].reshape(-1))
        for t in range(k): 
            model = ARIMA(history, order=(5,1,0))
            model_fit = model.fit(disp=0)
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            obs = test_y[t]
            history.append(obs)
            if(t%50==0):
                print('predicted=%f, expected=%f' % (yhat, obs))

        predictions = np.array(predictions)
        return predictions
    
    def accuracy(test, pred):
        test = test.reshape((-1,))
        pred = pred.reshape((-1,))
        error = mean_squared_error(test, pred)
        print('Test MSE: %.3f' % error)
        print("RMSE : %.3f"%(np.sqrt(error)))
        print("MAE : %.3f"%(mean_absolute_error(test,pred)))
        
    def pipeline(self, n_test=100, idx=0, train=False, load=True):
        self.load_data()
        if(train):
            # for i in range(self.n_members):
            #     model = self.fit_cnn_lstm()
            # filename = './drive/My Drive/ukdale' + str(self.eq-1) + '_model_' + str(i + 1) + '.h5'
            # model.save(filename)
            # print('>>> Saved %s' % filename)

            self.ann = self.fit_ANN()
            self.van_lstm = self.fit.fit_vanilla_lstm()
          
        if(load):
            self.members = self.load_all_models()
            print('Loaded %d models' % len(self.members))
        self.fit_stacked_model(inputX = self.val_x, inputy = self.val_y)
        self.make_cnnlstm_comparison_graph(n_test, idx)

In [0]:
d={}

In [0]:
# d={}
for i in range(15):
    d["obj{0}".format(i)] = Ensemble_pipeline(i+1, 3)
    if(i != 3 and i != 5):
        d["obj{0}".format(i)].pipeline(n_test = 500, train = False)
    elif(i == 5):
        d["obj{0}".format(i)].pipeline(n_test = 500, train = False, load = True)